In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from qiskit import Aer, transpile
from qiskit.algorithms import QAOA
from qiskit.algorithms.optimizers import COBYLA
from qiskit.utils import QuantumInstance
from qiskit_optimization import QuadraticProgram
from qiskit_optimization.algorithms import MinimumEigenOptimizer
from qiskit_optimization.converters import QuadraticProgramToQubo
from qiskit_optimization.translators import from_docplex_mp
from qiskit.circuit.library import TwoLocal
from docplex.mp.model import Model
import docplex.mp.solution as solution
import time
import warnings
warnings.filterwarnings('ignore')

class PortfolioOptimizer:
    """
    A class to implement Portfolio Optimization using QAOA as described in the paper
    "Portfolio Optimisation Using the D-Wave Quantum Annealer"
    """
    
    def __init__(self):
        """Initialize the PortfolioOptimizer"""
        self.backend = Aer.get_backend('qasm_simulator')
        
    def load_data(self, returns_file, cov_file=None):
        """
        Load returns and covariance data
        
        Parameters:
        returns_file (str): Path to CSV file with asset returns
        cov_file (str): Path to CSV file with covariance matrix (optional)
        
        Returns:
        tuple: (returns, covariance)
        """
        returns = pd.read_csv(returns_file, index_col=0)
        
        if cov_file:
            covariance = pd.read_csv(cov_file, index_col=0)
        else:
            # Calculate covariance matrix if not provided
            covariance = returns.cov()
            
        return returns, covariance
        
    def prepare_data(self, returns, covariance, N, asset_names=None):
        """
        Prepare data for optimization
        
        Parameters:
        returns (pd.DataFrame): Returns data
        covariance (pd.DataFrame): Covariance matrix
        N (int): Number of assets to consider
        asset_names (list): List of asset names to use (optional)
        
        Returns:
        tuple: (mu, sigma)
        """
        if asset_names:
            selected_assets = asset_names
        else:
            # If specific assets are not provided, take the first N
            selected_assets = returns.columns[:N]
            
        # Extract returns vector
        mu = returns[selected_assets].mean().values
        
        # Extract covariance matrix
        sigma = covariance.loc[selected_assets, selected_assets].values
        
        return mu, sigma
    
    def build_portfolio_qp(self, mu, sigma, n, R_star=0):
        """
        Build portfolio optimization quadratic program
        
        Parameters:
        mu (np.array): Expected returns vector
        sigma (np.array): Covariance matrix
        n (int): Number of assets to select
        R_star (float): Minimum required return
        
        Returns:
        QuadraticProgram: Qiskit quadratic program representation
        """
        N = len(mu)
        
        # Create a model using DOcplex
        mdl = Model("Portfolio Optimization")
        
        # Binary decision variables (select asset or not)
        x = mdl.binary_var_list(N, name="x")
        
        # Objective function: Minimize risk (x^T * Sigma * x)
        obj_expr = mdl.sum(sigma[i, j] * x[i] * x[j] for i in range(N) for j in range(N))
        mdl.minimize(obj_expr)
        
        # Constraint 1: Select n assets
        mdl.add_constraint(mdl.sum(x) == n)
        
        # Constraint 2: Meet or exceed target return
        if R_star > 0:
            mdl.add_constraint(mdl.sum(mu[i] * x[i] for i in range(N)) >= R_star)
            
        # Convert to Qiskit's QuadraticProgram
        qp = from_docplex_mp(mdl)
        
        return qp
    
    def compute_qubo_parameters(self, sigma, mu, n, R_star=0):
        """
        Compute appropriate QUBO parameters as described in the paper
        
        Parameters:
        sigma (np.array): Covariance matrix
        mu (np.array): Expected returns vector
        n (int): Number of assets to select
        R_star (float): Minimum required return
        
        Returns:
        tuple: (lambda_1, lambda_2)
        """
        N = len(mu)
        
        # Estimate lambda_1 as described in Section 4.3 of the paper
        max_benefit = 0
        for i in range(N):
            # Sort covariances of asset i with other assets
            sorted_covs = np.sort(sigma[i])
            # Sum the smallest n values
            benefit = np.sum(sorted_covs[:n])
            max_benefit = max(max_benefit, benefit)
        
        lambda_1 = max_benefit
        
        # Estimate lambda_2
        if R_star > 0:
            # Sort stocks by their returns
            sorted_indices = np.argsort(mu)
            
            # Calculate A1: average difference between smallest n sums
            smallest_n_stocks = sorted_indices[:n]
            sums = [np.sum(np.sort(sigma[i])[:n]) for i in smallest_n_stocks]
            A1 = np.mean(np.diff(np.sort(sums)))
            
            # Calculate A2: average positive difference in mu between these stocks
            A2 = np.mean(np.diff(np.sort(mu[smallest_n_stocks])))
            
            # Compute lambda_2
            lambda_2 = A1/A2 if A2 > 0 else 0.1
        else:
            lambda_2 = 0
            
        return lambda_1, lambda_2
    
    def run_qaoa(self, qp, p=3, shots=1024):
        """
        Run QAOA algorithm to solve the portfolio optimization problem
        
        Parameters:
        qp (QuadraticProgram): Qiskit quadratic program
        p (int): QAOA parameter p (number of layers)
        shots (int): Number of shots for quantum circuit execution
        
        Returns:
        tuple: (solution_dict, objective_value, execution_time)
        """
        start_time = time.time()
        
        # Convert to QUBO
        qubo_converter = QuadraticProgramToQubo()
        qubo = qubo_converter.convert(qp)
        
        # Set up quantum instance
        quantum_instance = QuantumInstance(
            backend=self.backend,
            shots=shots
        )
        
        # Set up QAOA
        qaoa_mes = QAOA(
            quantum_instance=quantum_instance,
            optimizer=COBYLA(maxiter=500),
            reps=p,
            initial_point=[0.01] * (2*p)
        )
        
        # Solve using MinimumEigenOptimizer
        meo = MinimumEigenOptimizer(qaoa_mes)
        result = meo.solve(qubo)
        
        # Get the solution
        solution_dict = {}
        for i, var in enumerate(qp.variables):
            solution_dict[var.name] = result.x[i]
        
        end_time = time.time()
        execution_time = end_time - start_time
        
        return solution_dict, result.fval, execution_time
    
    def run_classical_solver(self, qp):
        """
        Run classical solver for comparison
        
        Parameters:
        qp (QuadraticProgram): Qiskit quadratic program
        
        Returns:
        tuple: (solution_dict, objective_value, execution_time)
        """
        start_time = time.time()
        
        # Use the built-in classical solver
        classical_result = qp.solve()
        
        # Get the solution
        solution_dict = {}
        for i, var in enumerate(qp.variables):
            solution_dict[var.name] = classical_result.x[i]
        
        end_time = time.time()
        execution_time = end_time - start_time
        
        return solution_dict, classical_result.fval, execution_time
    
    def grid_search_parameters(self, mu, sigma, n, R_star=0, p=3):
        """
        Perform grid search to find the best lambda parameters
        
        Parameters:
        mu (np.array): Expected returns vector
        sigma (np.array): Covariance matrix
        n (int): Number of assets to select
        R_star (float): Minimum required return
        p (int): QAOA parameter p
        
        Returns:
        tuple: (best_lambda_1, best_lambda_2, best_solution)
        """
        # Get initial estimates
        lambda_1_hat, lambda_2_hat = self.compute_qubo_parameters(sigma, mu, n, R_star)
        
        # Create ranges for grid search (around the estimates)
        lambda_1_values = [lambda_1_hat * factor for factor in [0.5, 0.8, 1.0, 1.2, 1.5]]
        
        if R_star > 0:
            lambda_2_values = [lambda_2_hat * factor for factor in [0.5, 0.8, 1.0, 1.2, 1.5]]
        else:
            lambda_2_values = [0]
        
        best_fval = float('inf')
        best_lambda_1 = lambda_1_hat
        best_lambda_2 = lambda_2_hat
        best_solution = None
        
        for lambda_1 in lambda_1_values:
            for lambda_2 in lambda_2_values:
                # Build quadratic program with current lambdas
                qp = self.build_portfolio_qp(mu, sigma, n, R_star)
                
                # Convert to QUBO and update the lambda weights
                qubo_converter = QuadraticProgramToQubo()
                qubo = qubo_converter.convert(qp)
                
                # Run QAOA
                solution_dict, fval, _ = self.run_qaoa(qubo, p=p)
                
                # Check if this is the best solution
                if fval < best_fval:
                    best_fval = fval
                    best_lambda_1 = lambda_1
                    best_lambda_2 = lambda_2
                    best_solution = solution_dict
        
        return best_lambda_1, best_lambda_2, best_solution, best_fval
    
    def analyze_portfolio(self, solution_dict, mu, sigma):
        """
        Analyze portfolio solution
        
        Parameters:
        solution_dict (dict): Dictionary of variable assignments
        mu (np.array): Expected returns vector
        sigma (np.array): Covariance matrix
        
        Returns:
        dict: Portfolio analysis results
        """
        # Convert solution to array
        solution = np.array([solution_dict.get(f'x{i}', 0) for i in range(len(mu))])
        
        # Calculate portfolio return
        portfolio_return = np.dot(solution, mu)
        
        # Calculate portfolio risk
        portfolio_risk = np.dot(solution, np.dot(sigma, solution))
        
        # Identify selected assets
        selected_indices = np.where(solution == 1)[0]
        
        results = {
            'portfolio_return': portfolio_return,
            'portfolio_risk': portfolio_risk,
            'num_assets_selected': len(selected_indices),
            'selected_indices': selected_indices,
            'solution': solution
        }
        
        return results
    
    def process_index_data(self, index_name, N, n, R_star, p=3):
        """
        Process stock index data and run optimization
        
        Parameters:
        index_name (str): Name of the index (e.g., "Nikkei225", "S&P500")
        N (int): Number of assets to consider from index
        n (int): Number of assets to select
        R_star (float): Minimum required return
        p (int): QAOA parameter p
        
        Returns:
        dict: Results of optimization
        """
        # Load data (this would be your actual data loading procedure)
        print(f"Processing {index_name} index data...")
        print(f"Parameters: N={N}, n={n}, R*={R_star}, p={p}")
        
        # Generate synthetic data for demonstration
        np.random.seed(42)
        
        # Generate returns data (5-year returns as in the paper)
        mu = np.random.normal(0.05, 0.1, N)  # Average 5% return with some variability
        
        # Generate covariance matrix
        # First create a random correlation matrix
        A = np.random.normal(0, 1, size=(N, N))
        correlation = np.corrcoef(A)
        
        # Then convert to covariance using random volatilities
        volatilities = np.random.uniform(0.1, 0.3, N)
        sigma = np.outer(volatilities, volatilities) * correlation
        
        # Make sure covariance matrix is symmetric
        sigma = (sigma + sigma.T) / 2
        
        # Compute QUBO parameters
        lambda_1_hat, lambda_2_hat = self.compute_qubo_parameters(sigma, mu, n, R_star)
        print(f"Estimated parameters: lambda_1={lambda_1_hat:.2f}, lambda_2={lambda_2_hat:.2f}")
        
        # Build quadratic program
        qp = self.build_portfolio_qp(mu, sigma, n, R_star)
        
        # Run QAOA
        solution_dict, fval, qaoa_time = self.run_qaoa(qp, p=p)
        
        # Run classical solver for comparison
        classical_solution, classical_fval, classical_time = self.run_classical_solver(qp)
        
        # Analyze solutions
        qaoa_analysis = self.analyze_portfolio(solution_dict, mu, sigma)
        classical_analysis = self.analyze_portfolio(classical_solution, mu, sigma)
        
        # Print results
        print("\nResults:")
        print(f"QAOA Solution Objective Value: {fval:.2f}")
        print(f"QAOA Solution Time: {qaoa_time:.2f} seconds")
        print(f"QAOA Portfolio Return: {qaoa_analysis['portfolio_return']:.4f}")
        print(f"QAOA Portfolio Risk: {qaoa_analysis['portfolio_risk']:.4f}")
        
        print("\nClassical Solution Objective Value: {classical_fval:.2f}")
        print(f"Classical Solution Time: {classical_time:.2f} seconds")
        print(f"Classical Portfolio Return: {classical_analysis['portfolio_return']:.4f}")
        print(f"Classical Portfolio Risk: {classical_analysis['portfolio_risk']:.4f}")
        
        return {
            'qaoa': {
                'solution': solution_dict,
                'objective_value': fval,
                'time': qaoa_time,
                'analysis': qaoa_analysis
            },
            'classical': {
                'solution': classical_solution,
                'objective_value': classical_fval,
                'time': classical_time,
                'analysis': classical_analysis
            },
            'parameters': {
                'lambda_1': lambda_1_hat,
                'lambda_2': lambda_2_hat,
                'N': N,
                'n': n,
                'R_star': R_star
            }
        }
    
    def run_nikkei225_experiment(self):
        """Run experiment on Nikkei225 index data similar to paper"""
        results = []
        
        # Recreate a subset of the experiments from the paper
        experiments = [
            # N, n, R*
            (50, 10, 0),
            (50, 25, 0),
            (50, 10, 1200),
            (100, 20, 0),
            (100, 20, 2500)
        ]
        
        for N, n, R_star in experiments:
            result = self.process_index_data("Nikkei225", N, n, R_star)
            results.append(result)
            print("\n" + "="*50 + "\n")
            
        return results
    
    def run_sp500_experiment(self):
        """Run experiment on S&P500 index data similar to paper"""
        results = []
        
        # Recreate a subset of the experiments from the paper
        experiments = [
            # N, n, R*
            (100, 50, 3500),
            (200, 50, 3500)
        ]
        
        for N, n, R_star in experiments:
            result = self.process_index_data("S&P500", N, n, R_star)
            results.append(result)
            print("\n" + "="*50 + "\n")
            
        return results

def main():
    """Main function to run the portfolio optimization experiments"""
    optimizer = PortfolioOptimizer()
    
    print("Running Nikkei225 experiments...")
    nikkei_results = optimizer.run_nikkei225_experiment()
    
    print("\nRunning S&P500 experiments...")
    sp500_results = optimizer.run_sp500_experiment()
    
    # We'd typically save results here
    print("\nExperiments completed.")

if __name__ == "__main__":
    main()

ImportError: cannot import name 'Aer' from 'qiskit' (C:\Users\pablo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\qiskit\__init__.py)